<a href="https://colab.research.google.com/github/nidhid707/stock_pair/blob/master/DarvasBox.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [37]:
from google.colab import drive
drive.mount('/content/drive')


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [38]:
!pip install yfinance

In [44]:
Trade_collect=[]
Long_collect=[]

import yfinance as yf
import datetime
import pandas as pd
import numpy as np

ALLSTOCKLIST=pd.read_csv("/content/drive/My Drive/Colab Notebooks/ALLSTOCKLIST.csv")


for pair in ALLSTOCKLIST.index:
            nsedata = yf.Ticker(ALLSTOCKLIST["stock"][pair])
            stock1 = nsedata.history(period="3y",interval = "1d")
            stock1 = stock1[~stock1.index.duplicated(keep='first')]
            #stock1=stock1.reindex(index=stock1.index[::-1])
            stock1["Time"]=stock1.index
            stock1.Time = pd.to_datetime(stock1.Time, format='%Y-%m-%d %H:%M')
            #stock1 = stock1[['Time', 'Open', 'Close']]
            #stock1.columns = ['Time', 'Stock1_Open', 'Stock1_Close']
            stock1["stock1"]=ALLSTOCKLIST["stock"][pair]
            stock1["Trade"]=np.where(stock1["High"]>stock1["High"].shift(1),"Entry",np.where(stock1["Low"]<stock1["Low"].shift(1),"Exit","No Trade"))
            stock1["long_Entry"]=np.where((stock1["Trade"]=="Entry")&(stock1["Trade"].shift(1)=="Exit"),(stock1["Open"].shift(-1)),0)
            stock1["long_Exit"]=np.where((stock1["Trade"]=="Exit")&(stock1["Trade"].shift(1)=="Entry"),(stock1["Close"].shift(-1)),0)
            #Trade_collect.append(stock1)
            newdf=stock1[(stock1["long_Entry"]!=0)|(stock1["long_Exit"]!=0)]
            newdf=newdf[(((newdf["Trade"]=="Entry")&(newdf["Trade"].shift(1)=="Exit"))|(newdf["Trade"]=="Exit")&(newdf["Trade"].shift(1)=="Entry"))]
            newdf["Time"]=newdf["Time"].shift(1)
            newdf["long_Entry"]=np.where((newdf["long_Entry"]==0),(newdf["long_Entry"].shift(1)),0)
            newdf["Tstock"]=np.where(newdf["long_Entry"]!=0,100000/newdf["long_Entry"],0)
            newdf["TProfit"]=(newdf["long_Exit"]-newdf["long_Entry"])*newdf["Tstock"]
            newdf["Profit%"]=(newdf["TProfit"]/100000)*100
            
            print(newdf.tail(10))
            Long_collect.append(newdf.tail(10))


            
 

                    Open          High  ...       TProfit    Profit%
Date                                    ...                         
2020-10-26  11937.400391  11942.849609  ...   6718.973934   6.718974
2020-10-30  11678.450195  11748.950195  ...      0.000000   0.000000
2020-11-20  12813.400391  12892.450195  ...  10507.513471  10.507513
2020-11-23  12960.299805  12968.849609  ...      0.000000   0.000000
2020-11-26  12906.450195  13018.000000  ...   -258.789896  -0.258790
2020-11-27  13012.049805  13035.299805  ...      0.000000   0.000000
2020-12-10  13488.500000  13503.549805  ...   3457.682529   3.457683
2020-12-11  13512.299805  13579.349609  ...      0.000000   0.000000
2020-12-15  13547.200195  13589.650391  ...    819.735536   0.819736
2020-12-16  13663.099609  13692.349609  ...      0.000000   0.000000

[10 rows x 15 columns]
                  Open        High  ...       TProfit    Profit%
Date                                ...                         
2020-10-26  508.25

In [45]:
longt=pd.concat(Long_collect)
longt.to_csv('DarvasBox_result.csv')
from google.colab import files
files.download("DarvasBox_result.csv")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>